## Reinforcement Learning Project

**Studente:** Alessandro Mattei
**Matricola:** 295441
**Email:** alessandro.mattei1@student.univaq.it


In [1]:
from utils import *
import numpy as np
import pandas as pd

## Introduzione
### Il problema del cuoco


Consideriamo il caso in cui l'agente sia il vostro cuoco personale.

In particolare, l'agente (lo smiley sulla mappa) vuole cucinare la ricetta delle uova in base alle
indicazioni dell'utente (strapazzate o in un budino).

Per cucinare la ricetta desiderata, l'agente deve prima raccogliere gli strumenti necessari (il frullino
per le uova sulla mappa). Poi deve raggiungere i fornelli (la padella o il forno sulla mappa). Infine,
può cucinare.

Non che ci siano due speciali celle interconnesse (contrassegnate con 1a G) che permettono
all'agente di passare da un lato all'altro della mappa. Ma per farlo, l'agente deve esprimere la sua
volontà di andare dall'altra parte.

Le celle in (4, 2) e (9, 3) sono i cancelli speciali. Permettono all'agente di passare da un lato
all'altro della mappa. Queste due celle speciali sono collegate tra loro, ma l'agente deve esprimere
la sua volontà di andare dall'altra parte.

Dato che siete molto affamati, è fondamentale che l'agente cucini le uova secondo i vostri gusti
(strapazzate/pudding) il più velocemente possibile senza farvi aspettare più del necessario.

![Figura 1](imgs/figura_1.png "figura 1")

Figura 1: Un'istanza particolare del problema di cottura di Chef. L'obiettivo è che l'agente
attualmente situato nello stato (4, 3) abbia una politica che porti sempre a cucinare le uova nella
posizione (1, 4) o (8, 4). Le celle in (4, 2) e (9, 3) sono quelle del cancello speciale.

Consideriamo l'istanza mostrata nella Figura 1:
- Nella figura, l'agente si trova in (4, 3) (ma può iniziare in qualsiasi cella della griglia).
- L'agente ha bisogno di strumenti di cottura come il frullino per le uova in posizione (1, 3) e (8,3).
- Ci sono due obiettivi finali diversi, visualizzati come la padella in posizione (1, 4) e il forno in posizione (8, 4).
- Le celle in (4, 2) e (9, 3) sono i cancelli speciali. Permettono all'agente di passare da un lato all'afro della mappa. Queste due celle speciali sono collegate tra loro, ma l'agente deve esprimere la sua volontà di andare dall'altra parte.
- L'agente non può muoversi in diagonale.
- Le pareti sono rappresentate da linee nere spesse.
- L'agente non può muoversi attraverso i muri.
- Un episodio terminerà quando l'agente riuscirà a cucinare le uova strapazzate (vedi descrizione precedente).

## Analisi

Per semplificare il problema consideriamo separatamente gli stati in cui l'agente non hanno in mano lo sbattitore da quelli che ha lo sbattitore (situato in posizione **(3,1)** e **(3.8)** visibile nelle foto in basso). 
Per far ciò ho duplicato la griglia.

![Figura 2](imgs/figura_2.png "figura 2")

Figura 2: prima griglia che contiene solo gli sbattitori

Nella prima griglia (nella figura sopra), l'agente ha l'obiettivo di raggiungere uno dei due **sbattitori**.

![Figura 3](imgs/figura_3.png "figura 3")

Figura 3: seconda griglia che contiene solo le postazioni di cottura

![Figura 4](imgs/figura_4.png "figura 4")

Figura 4: griglia complessiva. Una volta che l'agente avrà preso uno sbattitore potrà andare nella griglia 2 per cucinare (come indicato dalle frecce).

Non appena l'agente esegue l'azione **take_whiskr** da una delle due posizioni contenenti uno **sbattitore**, esso viene portato nella seconda griglia seguendo le frecce mostrate nella **figura 4**:
- Se l'agente prende lo sbattitore in posizione **(3, 1)** nella griglia in basso, esso viene portato nella griglia in alto in posizione **(7, 1)**
- Se l'agente prende lo sbattitore in posizione **(3, 8)** nella griglia in basso, esso viene portato nella griglia in alto in posizione **(7, 8)**

Dopo aver raggiunto e **preso uno sbattitore**, l'agente deve raggiungere una delle due postazioni di cucina che sono nella griglia superiore (situati nelle posizioni **(8, 1)** e **(8, 8)** nella griglia in alto) per cucinare le uova (la postazione di cucina da raggiungere dipende dalla ricetta che l'agente deve cucinare)

L'agente dovrà percorrere entrambe le griglie per raggiungere l'obiettivo finale. Dalla prima griglia alla seconda griglia.

Nella seconda griglia (griglia situata in alto), l'agente ha l'obiettivo di raggiungere una delle due postazioni di cucina e di eseguire quindi l'azione **cook**:
- la postazione di cucina per il piatto **scrambled eggs** è situato in posizione **(8, 1)**
- la postazione di cucina per il piatto **pudding eggs** è situato in posizione **(8, 8)**

In entrambe le griglie sono situati dei **Gate** che hanno la funzione di trasportare l'agente da una stanza all'altra delle stessa griglia, l'agente quando si trova su uno di questi può usare il **Gate** tramite l'azione **go_rigth** o **go_left**:
- la postazione di **Gate** situato in posizione **(2, 4)** trasporta l'agente in posizione **(3, 9)** se effettua l'azione **go_right**
- la postazione di **Gate** situato in posizione **(3, 9)** trasporta l'agente in posizione **(2, 4)** se effettua l'azione **go_left**
- la postazione di **Gate** situato in posizione **(6, 4)** trasporta l'agente in posizione **(7, 9)** se effettua l'azione **go_right**
- la postazione di **Gate** situato in posizione **(7, 9)** trasporta l'agente in posizione **(6, 4)** se effettua l'azione **go_left**

#### Definiamo tutti i possibili stati

Ogni stato del sistema corrisponde a una posizione possibile che l'agente può assumere nella griglia.

Tutte le posizione della **colonna 5** che non possono essere fisicamente raggiunte non saranno presenti tra gli stati possibili del problema (posizioni impossibili).

Vediamo una rappresentazione in Tupla posizionale.

In [2]:
print_states(dict_view=True)

 (8, 1)  (8, 2)  (8, 3)  (8, 4)          (8, 6)  (8, 7)  (8, 8)  (8, 9) 
 (7, 1)  (7, 2)  (7, 3)  (7, 4)          (7, 6)  (7, 7)  (7, 8)  (7, 9) 
 (6, 1)  (6, 2)  (6, 3)  (6, 4)          (6, 6)  (6, 7)  (6, 8)  (6, 9) 
 (5, 1)  (5, 2)  (5, 3)  (5, 4)          (5, 6)  (5, 7)  (5, 8)  (5, 9) 


 (4, 1)  (4, 2)  (4, 3)  (4, 4)          (4, 6)  (4, 7)  (4, 8)  (4, 9) 
 (3, 1)  (3, 2)  (3, 3)  (3, 4)          (3, 6)  (3, 7)  (3, 8)  (3, 9) 
 (2, 1)  (2, 2)  (2, 3)  (2, 4)          (2, 6)  (2, 7)  (2, 8)  (2, 9) 
 (1, 1)  (1, 2)  (1, 3)  (1, 4)          (1, 6)  (1, 7)  (1, 8)  (1, 9) 


Vediamo una rappresentazione dei stati in versione numerica.

In [3]:
print_states()

 56  57  58  59      60  61  62  63 
 48  49  50  51      52  53  54  55 
 40  41  42  43      44  45  46  47 
 32  33  34  35      36  37  38  39 


 24  25  26  27      28  29  30  31 
 16  17  18  19      20  21  22  23 
 8   9   10  11      12  13  14  15 
 0   1   2   3       4   5   6   7  


Il numero totale di Stati è 64.

In [4]:
cooking_chef_states_tuples, cooking_chef_tuples_states = create_problem_grid()
S = np.array(list(cooking_chef_states_tuples.keys()), dtype=np.int64)
S

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63], dtype=int64)

## Definizioni delle azioni Possibili

L'agente può effettuare le **azioni di movimento** (**left**, **right**, **up**, **down**) e inoltre può effettuare alcune azioni specifiche per specifichi stati:
- L'azione **take_whiskr** corrisponde all'azione di prendere lo sbattitore. A seguito della sua applicazione, l'agente viene portato nella seconda griglia.
- L'azione **go_right** corrisponde all'azione di utilizzo del Gate posizionato nella stanza di sinistra. A seguito della sua applicazione, l'agente viene portato nella seconda stanza dalle prima stanza.
- L'azione **go_left** corrisponde all'azione di utilizzo del Gate posizionato nella stanza di destra. A seguito della sua applicazione, l'agente viene portato nella prima stanza dalla seconda stanza.
- L'azione **cook** corrisponde all'azione di cucinare le uova. In questo caso, l'applicazione di questa azione porta l'agente a rimanere fermo (absorbing state). L'esecuzione di questa azione rappresenta l'obiettivo finale dell'agente

L'insieme **A** è il seguente:
- azione 0: **left ←**
- azione 1: **right →**
- azione 2: **up ↑**
- azione 3: **down ↓**
- azione 4: **take_whiskr T**
- azione 5: **go_right R**
- azione 6: **go_left L**
- azione 7: **cook C**

In [5]:
left = 0
right = 1
up = 2
down = 3
take_whiskr = 4
go_right = 5
go_left = 6
cook = 7

actions = [left, right, up, down, take_whiskr, go_right, go_left, cook]

A = np.array(actions, dtype=np.int64)
A

array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int64)

Costruiamo ora una mappa delle azioni possibili per ogni stato.
In questo caso modelliamo questa mappa per permettere entrambe le ricette contemporaneamente, ma in seguito se decidiamo una ricetta elimineremo l'azione di cucinare in funzione della ricetta:
-  Se si decide di cucinare **scrambled eggs (8, 1)** allora l'azione **cook** allo stato **62 (8,8)** verrà eliminata dalla mappa
-  Se si decide di cucinare **pudding eggs (8,8)** allora l'azione **cook** allo stato **56 (8,1)** verrà eliminata dalla mappa

Un'altra alternativa alla rimozione dell'azione **cook** è quella di impostare una ricompensa estremamente negativa e svantaggiosa, così da non farla prediligere rispetto alle altre azioni possibili.

In [6]:
states_actions_mapping = {
    0: [1],
    1: [0, 1],
    2: [0, 1],
    3: [0, 2],
    4: [1, 2],
    5: [0, 1, 2],
    6: [0, 1],
    7: [0, 2],
    8: [1, 2],
    9: [0, 1],
    10: [0, 1],
    11: [0, 2, 3, 5],
    12: [1, 2, 3],
    13: [0, 2, 3],
    14: [1, 2],
    15: [0, 2, 3],
    16: [3, 4],
    17: [1, 2],
    18: [0, 1, 2],
    19: [0, 2, 3],
    20: [1, 2, 3],
    21: [0, 2, 3],
    22: [1, 3, 4],
    23: [0, 3, 6],
    24: [1],
    25: [0, 1, 3],
    26: [0, 1, 3],
    27: [0, 3],
    28: [1, 3],
    29: [0, 1, 3],
    30: [0, 1],
    31: [0],

    32: [1],
    33: [0, 1],
    34: [0, 1],
    35: [0, 2],
    36: [1, 2],
    37: [0, 1, 2],
    38: [0, 1],
    39: [0, 2],
    40: [1, 2],
    41: [0, 1],
    42: [0, 1],
    43: [0, 2, 3, 5],
    44: [1, 2, 3],
    45: [0, 2, 3],
    46: [1, 2],
    47: [0, 2, 3],
    48: [3],
    49: [1, 2],
    50: [0, 1, 2],
    51: [0, 2, 3],
    52: [1, 2, 3],
    53: [0, 2, 3],
    54: [1, 3],
    55: [0, 3, 6],
    56: [1, 7],
    57: [0, 1, 3],
    58: [0, 1, 3],
    59: [0, 3],
    60: [1, 3],
    61: [0, 1, 3],
    62: [0, 1, 7],
    63: [0]
}

mapping = convert_to_numpy_array(states_actions_mapping, S, A);

## Creazione insieme di Transizioni P 
### d) Report the transition function $P$ for an state $s$ and action $a$ in a tabular format.

Creiamo la matrice a 3 dimensioni di transizione $P[A, S, S']$:
- $A$: sono le Azioni
- $S$: sono gli stati sorgente
- $S'$: sono gli stati di destinazione

Modelliamo il problema assumendo che non ci sono effetti imprevisti a seguito dell'applicazione delle azioni:

da uno stato **sorgente** applicando l'**azione**, abbiamo soltanto **uno stato destinazione** (stato raggiungibile con probabilità **1**)

![Figura 5](imgs/figura_5.png "figura 5")
Figura 5: Esempio di comportamento dell'agente a seguito dell'applicazione dell'azione **up**

In [7]:
def next_state(s: int, a: int):
    state_tuple = cooking_chef_states_tuples[s]
    tile = state_tuple
    if a == 0 and a in states_actions_mapping[s]:
        # azione left
        tile = (state_tuple[0], state_tuple[1] - 1)
    elif a == 1 and a in states_actions_mapping[s]:
        # azione right
        tile = (state_tuple[0], state_tuple[1] + 1)
    elif a == 2 and a in states_actions_mapping[s]:
        # azione up
        tile = (state_tuple[0] + 1, state_tuple[1])
    elif a == 3 and a in states_actions_mapping[s]:
        # azione down
        tile = (state_tuple[0] - 1, state_tuple[1])
    elif a == 4 and a in states_actions_mapping[s]:
        # azione take_whiskr
        if s == 16:
            # azione take_whiskr dallo stato 16 (16 -> 48)
            tile = cooking_chef_states_tuples[48]
        elif s == 22:
            # azione take_whiskr dallo stato 22 (22 -> 54)
            tile = cooking_chef_states_tuples[54]
        else:
            raise RuntimeError("Azione take_whiskr non valida")
    elif a == 5 and a in states_actions_mapping[s]:
        # azione go_right
        if s == 11:
            # azione go_right dallo stato 11 (11 -> 23)
            tile = cooking_chef_states_tuples[23]
        elif s == 43:
            # azione go_right dallo stato 43 (43 -> 55)
            tile = cooking_chef_states_tuples[55]
        else:
            raise RuntimeError("Azione go_right non valida")
    elif a == 6 and a in states_actions_mapping[s]:
        # azione go_left
        if s == 23:
            # azione go_left dallo stato 23 (23 -> 11)
            tile = cooking_chef_states_tuples[11]
        elif s == 55:
            # azione go_left dallo stato 55 (55 -> 43)
            tile = cooking_chef_states_tuples[43]
        else:
            raise RuntimeError("Azione go_left non valida")
    elif a == 7 and a in states_actions_mapping[s]:
        # azione cook
        tile = state_tuple
    else:
        # azione non possibile
        tile = state_tuple

    return cooking_chef_tuples_states[tile]


def get_possible_actions_from_state(s: int):
    actions = []
    for a in A:
        if mapping[s, a] > 0.0:
            actions.append(a)
    return actions


def generate_transitions(
        actions: np.ndarray, states: np.ndarray
):
    transitions = np.zeros(
        shape=(len(actions), len(states), len(states)), dtype=np.float64
    )

    for state in states:
        for action in get_possible_actions_from_state(state):
            if mapping[state, action] > 0.0:
                # l'azione `action` è possibile dallo stato `state`
                # non ci sono effetti imprevisti, c'è solo uno stato destinazione
                # raggiungibile con probabilità 1
                transitions[action, state, next_state(state, action)] = 1

    return transitions


P = generate_transitions(A, S)

In [8]:
next_state(0, 0)

0

Costruiamo le varie matrici di transizione $P$ per ogni azione specificata.

Stampiamo una tabella che descrive una matrice $S X S'$ per l'azione scelta.
$P[left, S, S']$

In [9]:
transition_left = print_transitions_matrix(p=P, action=left)
transition_left.to_csv("transitions/transition_left.csv")
transition_left

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


<p>
$P[right, S, S']$
</p>

In [10]:
transition_right = print_transitions_matrix(p=P, action=right)
transition_right.to_csv("transitions/transition_right.csv")
transition_right

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


<p>$P[up, S, S']$</p>

In [11]:
transition_up = print_transitions_matrix(p=P, action=up)
transition_up.to_csv("transitions/transition_up.csv")
transition_up

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>$P[down, S, S']$</p>

In [12]:
transition_down = print_transitions_matrix(p=P, action=down)
transition_down.to_csv("transitions/transition_down.csv")
transition_down

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>$P[take_whiskr, S, S']$</p>

In [13]:
transition_take_whiskr = print_transitions_matrix(p=P, action=take_whiskr)
transition_take_whiskr.to_csv("transitions/transition_take_whiskr.csv")
transition_take_whiskr

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>$P[go_right, S, S']$</p>

In [14]:
transition_go_right = print_transitions_matrix(p=P, action=go_right)
transition_go_right.to_csv("transitions/transition_go_right.csv")
transition_go_right

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>$P[go_left, S, S']$</p>

In [15]:
transition_go_left = print_transitions_matrix(p=P, action=go_left)
transition_go_left.to_csv("transitions/transition_go_left.csv")
transition_go_left

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<p>$P[cook, S, S']$</p>

In [16]:
transition_cook = print_transitions_matrix(p=P, action=cook)
transition_cook.to_csv("transitions/transition_cook.csv")
transition_cook

,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,...,s54,s55,s56,s57,s58,s59,s60,s61,s62,s63
Stati,,,,,,,,,,,,,,,,,,,,,
s0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s60,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
s61,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Creazione Insieme di reward $R$
### e) Describe a reward function $R : S × A × S$ and a value of $γ$ that will lead to an optimal policy

Il nostro obbiettivo è cucinare il piatto **pudding eggs**.

Un possibile approccio alla scelta dei rewards da assegnare all'agente per guidarlo nella risoluzione del problema è il seguente:
- Premio per l'obbiettivo $+100$ cucinare il piatto **pudding eggs** allo stato 62
- Premio per l'obbiettivo intermedio $+50$ prendere uno sbattitore posizionato allo stato **16** o **22**
- Penalizzazione per un azione non valida $-1$
- nei casi rimanenti il Premio sarà $0$

In [17]:
R = np.zeros(shape=(len(S), len(A), len(S)), dtype=np.float64)
# Obbiettivo
R[62, cook, 62] = 100
R[56, cook, 56] = -1

R[16, take_whiskr, 48] = 50
R[22, take_whiskr, 54] = 50


def probs(a: int, s: int):
    return mapping[s, a]


for s in S:
    for a in A:
        # settiamo il reward a -1 se l'azione `a` non è valida
        # nello stato `s`
        # (ovvero quando non è possibile eseguire l'azione
        # `a` dallo stato `s`)
        if probs(a, s) == 0.0:
            R[s, a, s] = -1

## implementazione scenario Model free

### h) Now, considering the problem as a model-free scenario, provide a program (written in Python, possibly based on the labs) that can compute the optimal policy for this world by solely considering the pudding eggs scenario. Draw the computed policy in the grid by putting the optimal action in each cell. If multiple actions are possible, include the probability of each arrow. There may be multiple optimal policies; pick one to show it. Note that the model is not available for computation but must be encoded to be used as the "real-world" environment.

In questo caso, non sarà possibile accedere al **transition model** e al **reward model** del problema.

Per la specifica **Model-free**, consideriamo soltanto lo scenario **pudding eggs**.

Definiamo una funzione di policy iniziale insieme a una funzione di generazione di episodi. Useremo e adatteremo il codice sviluppato durante il laboratorio.

In [74]:
def simple_policy(s: int):
    return states_actions_mapping[s][0]

STARTING_STATE_ID = 19
DISCOUNT_FACTOR = 0.5 
starting_state = S[STARTING_STATE_ID]
states = [starting_state]
policy = {}
policy_function = simple_policy

for action in A:
    policy[(starting_state, action)] = 1 if action == policy_function(starting_state) else 0

current_state = starting_state

def generate_episode(length = 200, return_actions=False, first_state=None, first_action=None, random_first_state=False):
    state = starting_state if first_state == None else first_state
    if random_first_state:
        initial_states = states[states < 32]
        state = np.random.choice(initial_states)

    action_state = []
    episode_state = [state]

    if first_action != None:
        action_state.append(first_action)
        state = next_state(state, first_action)
        episode_state.append(state)

    while len(episode_state) < length:
        if state not in states:
            states.append(state)
            for action in A:
                policy[(state, action)] = 0
            policy[(state, policy_function(state))] = 1
        
        action_probabilities = [policy[(state, action)] for action in A]
        action = np.random.choice(list(A), p=action_probabilities)
        action_state.append(action)
        state = next_state(state, action)
        episode_state.append(state)
        
        if state == 62 and action == 7:
            break
            
    action_probabilities = [policy[(current_state, action)] for action in A]
    action = np.random.choice(list(A), p=action_probabilities)
    action_state.append(action)
    
    state_action = []
    for i in range(len(episode_state)):
        state_action.append({'s': episode_state[i], 'a': action_state[i]})
        
    
    return episode_state if not return_actions else state_action


In [75]:
generate_episode()

[19,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18,
 17,
 18]

Definiamo una funzione che ci calcola una policy ottima, utilizzeremo la funzione vista in laboratorio.

In [76]:
def compute_returns(episode, index):
    if index == None:
        return 0
    episode = episode[index:]
    sum_returns = 0
    for i in range(0,len(episode)):
        sum_returns += R[episode[i]['s'], episode[i]['a'], next_state(episode[i]['s'], episode[i]['a'])] * (DISCOUNT_FACTOR ** i)

    return sum_returns

def get_easy_policy():
    pol = {}
    for k,v in policy.items():
        if v != 0:
            pol[k[0]] = k[1]
    return pol

def get_state_action_from_episodes(episodes):
    episode_state = []
    action_state = []
    for e in episodes:
        episode_state.append(e['s'])
        action_state.append(e['a'])
    return (episode_state, action_state)

def improve_policy():
    n = {(state, action): 0 for state in states for action in A}
    g = {(state, action): 0 for state in states for action in A}
    q = {(state, action): 0 for state in states for action in A}
    known_states = [state for state in states]
    i=0
    for i in range(10):
        print(i)
        i+=1
        for state in states: #63
            for action in A: #7
                epi = generate_episode(return_actions=True, first_state = state, first_action = action) 
                episode, actions = get_state_action_from_episodes(epi)

                #check if all states are known
                for state_no in episode: #200
                    if state_no not in known_states:
                        known_states.append(state_no)
                        for action in A: #7
                            n[(state_no, action)] = 0
                            g[(state_no, action)] = 0
                            q[(state_no, action)] = 0 # 6.174.000

                ## COMPUTATION OF Q-VALUES
                for j in range(len(episode)): # 200
                    n[(episode[j], actions[j])] += 1
                    g[(episode[j], actions[j])] += compute_returns(epi, j)
                    q[(episode[j], actions[j])] = g[(episode[j], actions[j])]/n[(episode[j], actions[j])] # 882.000
                    

    print(q)
    ## POLICY IMPROVEMENT
    
    for state_no,ac in q.keys():

        if state_no not in [s for s in states]:
           states.append(state_no)

        max_action = None
        max_value = -np.inf
        for action in A:
            if q[(state_no, action)] > max_value:
                max_value = q[(state_no, action)]
                max_action = action


        newpolicy = policy
        for action in A:
            newpolicy[(state_no, action)] = 1 if action == max_action else 0

        policy = newpolicy

    return policy

In [41]:
asdf  = []
for i in range(5):
    asdf.append({'s': i, 'a': i})

print(asdf[1])

sdf = []    
sdfg = []
for a in asdf:
    sdf.append(a['s'])
    sdfg.append(a['a'])
print(sdfg)
sdf

{'s': 1, 'a': 1}
[0, 1, 2, 3, 4]


[0, 1, 2, 3, 4]

In [77]:
improve_policy()

0
1
2
3
4
5
6
7
8
9
{(19, 0): -2.212627654350628e-60, (19, 1): -1.0, (19, 2): 0.0, (19, 3): 0.0, (19, 4): -1.0, (19, 5): -1.0, (19, 6): -1.0, (19, 7): -1.0, (18, 0): -0.004472396925227082, (18, 1): -1.2446030555722283e-60, (18, 2): -1.2446030555722283e-60, (18, 3): -1.0, (18, 4): -1.0, (18, 5): -1.0, (18, 6): -1.0, (18, 7): -1.0, (17, 0): -1.0, (17, 1): -0.0022513015336991533, (17, 2): 0.0, (17, 3): -1.0, (17, 4): -1.0, (17, 5): -1.0, (17, 6): -1.0, (17, 7): -1.0, (27, 0): -1.3828922839691425e-61, (27, 1): -1.0, (27, 2): -1.0, (27, 3): -1.2446030555722283e-60, (27, 4): -1.0, (27, 5): -1.0, (27, 6): -1.0, (27, 7): -1.0, (26, 0): -1.1713911111268031e-60, (26, 1): 0.0, (26, 2): -1.0, (26, 3): 0.0, (26, 4): -1.0, (26, 5): -1.0, (26, 6): -1.0, (26, 7): -1.0, (25, 0): -0.003357465113839027, (25, 1): -1.2446030555722283e-60, (25, 2): -1.0, (25, 3): -1.2446030555722283e-60, (25, 4): -1.0, (25, 5): -1.0, (25, 6): -1.0, (25, 7): -1.0, (24, 0): -1.0, (24, 1): -0.0016915107305211844, (24, 2): -1.0

UnboundLocalError: cannot access local variable 'policy' where it is not associated with a value